#### **Welcome to Assignment 2 on Deep Learning for Computer Vision.**
This assignment consists of three parts. Part-1 is based on the content you learned in Week-3 of course and Part-2 is based on the content you learned in Week-4 of the course. Part-3 is **un-graded** and mainly designed to help you flex the Deep Learning muscles grown in Part-2. 

Unlike the first two parts, you'll have to implement everything from scratch in Part-3. If you find answers to questions in Part-3, feel free to head out to the forums and discuss them with your classmates!

#### **Instructions**
1. Use Python 3.x to run this notebook
2. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'.
you should not change anything else in the code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
3. Read documentation of each function carefully.
4. All the Best!

# Part-1

In [ ]:
# Imports
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# %matplotlib inline uncomment this line if you're running this notebook on your local PC

In [ ]:
# DO NOT CHANGE THIS CODE
np.random.seed(10)

### Question 1 : Line detection from a given image.


Find the starting and ending point co-ordinates of detected lines of a given image using hough transform. 

Following criterion need to be satisfied to qualify as a line:

(a) Minimum line length = 60;
(b) Maximum allowed gap between line segments = 250;
(c) Accumulator threshold parameter = 15  (only those lines are returned that get enough votes);
(d) Distance resolution of the accumulator in pixels = 1;
(e) Angle resolution of the accumulator in radians = pi/180


Which of the following pairs of (start, end) co-ordinates of detected lines are true:

1. line-1: (1,81), (27,9) ; line-2 : (92,0), (95,64)
2. line-1: (2,79), (26,11) ; line-2 : (92,1), (94,67)
3. line-1: (2,80), (26,10) ; line-2: (91,2), (96,65)
4.  None of the above

In [ ]:
#Read image 
img = cv2.imread('line.png', cv2.IMREAD_COLOR)

# Visualize the input image
plt.imshow(img)
plt.title('Input Image')
plt.show()


#convert the image to gray-scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Find the edges in the image using canny detector

edges = cv2.Canny(gray, 50, 200)
#plt.imshow(edges)
#plt.show()

####YOUR CODE STARTS HERE #####
rho = 1
theta = np.pi/180
threshold = 15
min_line_length = 60
max_line_gap = 250
line_image = np.copy(img)*0 #creating a blank to draw lines on

lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)

for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image, (x1,y1), (x2,y2), (255,0,0), 5)
    
img_line = cv2.addWeighted(img, 0.8, line_image, 1, 0)

####YOUR CODE ENDS HERE #####
plt.imshow(img_line)
plt.title('Detected Line Image')
plt.show()

print(lines)

##Question 2: Point matching using RANSAC

Given two sets of points related by affine transformation(with an outlier rate), use the RANSAC method to estimate the Affine transformation parameters between them and the number of inliers(Matching points).

Which of the following is the estimated number of inliers for an outlier rate of 0.9:

1. 89
2. 101
3. 117
4. 132

In [ ]:
import numpy as np
np.random.seed(0)


# Affine Transform
# |x'|  = |a, b| * |x|  +  |tx|
# |y'|    |c, d|   |y|     |ty|
# pts_t =    A   * pts_s  + t

# -------------------------------------------------------------
# Test Class Affine

class Affine_Transform():

    def create_test_case(self, outlier_rate=0):
        ''' CREATE_TEST_CASE

            Randomly generate a test case of affine transformation.

            Input arguments:

            - outlier_rate : the percentage of outliers in test case,
            default is 0

            Outputs:

            - pts_s : Source points that will be transformed
            - pts_t : warped points
            - A, t : parameters of affine transformation, A is a 2x2
            matrix, t is a 2x1 vector, both of them are created randomly

        '''

        # Randomly generate affine transformation
        # A is a 2x2 matrix, the range of each value is from -2 to 2
        A = 4 * np.random.rand(2, 2) - 2

        # % t is a 2x1 VECTOR, the range of each value is from -10 to 10
        t = 20 * np.random.rand(2, 1) - 10

        # Set the number of points in test case
        num = 1000

        # Compute the number of outliers and inliers respectively
        outliers = int(np.round(num * outlier_rate))
        inliers = int(num - outliers)

        # Gernerate source points whose scope from (0,0) to (100, 100)
        pts_s = 100 * np.random.rand(2, num)
        # Initialize warped points matrix
        pts_t = np.zeros((2, num))

        # Compute inliers in warped points matrix by applying A and t
        pts_t[:, :inliers] = np.dot(A, pts_s[:, :inliers]) + t

        # Generate outliers in warped points matrix
        pts_t[:, inliers:] = 100 * np.random.rand(2, outliers)

        # Reset the order of warped points matrix,
        # outliers and inliers will scatter randomly in test case
        rnd_idx = np.random.permutation(num)
        pts_s = pts_s[:, rnd_idx]
        pts_t = pts_t[:, rnd_idx]

        return A, t, pts_s, pts_t

    def estimate_affine(self, pts_s, pts_t):
        ''' ESTIMATE_AFFINE

            Estimate affine transformation by the given points
            correspondences.

            Input arguments:
            - pts_t : points in target image
            - pts_s : points in source image

            Outputs:

            - A, t : the affine transformation, A is a 2x2 matrix
            that indicates the rotation and scaling transformation,
            t is a 2x1 vector determines the translation

            Method:

            To estimate an affine transformation between two images,
            at least 3 corresponding points are needed.
            In this case, 6-parameter affine transformation are taken into
            consideration, which is shown as follows:

            | x' | = | a b | * | x | + | tx |
            | y' |   | c d |   | y |   | ty |

            For 3 corresponding points, 6 equations can be formed as below:

            | x1 y1 0  0  1 0 |       | a  |       | x1' |
            | 0  0  x1 y1 0 1 |       | b  |       | y1' |
            | x2 y2 0  0  1 0 |   *   | c  |   =   | x2' |
            | 0  0  x2 y2 0 1 |       | d  |       | y2' |
            | x3 y3 0  0  1 0 |       | tx |       | x3' |
            | 0  0  x3 y3 0 1 |       | ty |       | y3' |

            |------> M <------|   |-> theta <-|   |-> b <-|

            Solve the equation to compute theta by:  theta = M \ b
            Thus, affine transformation can be obtained as:

            A = | a b |     t = | tx |
                | c d |         | ty |

        '''

        # Get the number of corresponding points
        pts_num = pts_s.shape[1]

        # Initialize the matrix M,
        # M has 6 columns, since the affine transformation
        # has 6 parameters in this case
        M = np.zeros((2 * pts_num, 6))

        for i in range(pts_num):
            # Form the matrix m
            temp = [[pts_s[0, i], pts_s[1, i], 0, 0, 1, 0],
                    [0, 0, pts_s[0, i], pts_s[1, i], 0, 1]]
            M[2 * i: 2 * i + 2, :] = np.array(temp)

        # Form the matrix b,
        # b contains all known target points
        b = pts_t.T.reshape((2 * pts_num, 1))

        try:
            # Solve the linear equation
            theta = np.linalg.lstsq(M, b)[0]

            # Form the affine transformation
            A = theta[:4].reshape((2, 2))
            t = theta[4:]
        except np.linalg.linalg.LinAlgError:
            # If M is singular matrix, return None
            # print("Singular matrix.")
            A = None
            t = None

        return A, t
# -------------------------------------------------------------

# Create instance
af = Affine_Transform()

# Generate a test case as validation with
# a rate of outliers
outlier_rate = 0.90
A_true, t_true, pts_s, pts_t = af.create_test_case(outlier_rate)

# At least 3 corresponding points to
# estimate affine transformation
K = 3
# Randomly select 3 pairs of points to do estimation
idx = np.random.randint(0, pts_s.shape[1], (K, 1))

A_test, t_test = af.estimate_affine(pts_s[:, idx], pts_t[:, idx])

# Display known parameters with estimations
# They should be same when outlier_rate equals to 0,
# otherwise, they are totally different in some cases
#print(A_true, '\n', t_true)
#print(A_test, '\n', t_test)

# -------------------------------------------------------------
# Test Class Ransac
# The number of iterations in RANSAC
ITER_NUM = 2000


class Ransac():

    def __init__(self, K=3, threshold=1):
        ''' __INIT__

            Initialize the instance.

            Input argements:

            - K : the number of corresponding points,
            default is 3
            - threshold : determing which points are inliers
            by comparing residual with it

        '''

        self.K = K
        self.threshold = threshold

    def residual_lengths(self, A, t, pts_s, pts_t):
        ''' RESIDUAL_LENGTHS

            Compute residual length (Euclidean distance) between
            estimation and real target points. Estimation are
            calculated by the given source point and affine
            transformation (A & t).

            Input arguments:

            - A, t : the estimated affine transformation calculated
            by least squares method
            - pts_s : key points from source image
            - pts_t : key points from target image

            Output:

            - residual : Euclidean distance between estimated points
            and real target points

        '''

        if not(A is None) and not(t is None):
            # Calculate estimated points:
            # pts_e = A * pts_s + t
            pts_e = np.dot(A, pts_s) + t

            # Calculate the residual length between estimated points
            # and target points
            diff_square = np.power(pts_e - pts_t, 2)
            residual = np.sqrt(np.sum(diff_square, axis=0))
        else:
            residual = None

        return residual

    def ransac_fit(self, pts_s, pts_t):
        ''' RANSAC_FIT

            Apply the method of RANSAC to obtain the estimation of
            affine transformation and inliers as well.

            Input arguments:

            - pts_s : key points from source image
            - pts_t : key points from target image

            Output:

            - A, t : estimated affine transformation
            - inliers : indices of inliers that will be applied to refine the
            affine transformation

        '''
        #### YOUR CODE START HERE


        
        #### YOUR CODE ENDS HERE
# -------------------------------------------------------------

# Create instance
rs = Ransac(K=3, threshold=1)

residual = rs.residual_lengths(A_test, t_test, pts_s, pts_t)

# Run RANSAC to estimate affine transformation when
# too many outliers in points set
A_rsc, t_rsc, inliers = rs.ransac_fit(pts_s, pts_t)

# print the number of inliners or point matches
print (inliers[0].shape)


### Question 3: Detect corners in a given image using Harris Corner Detection Algorithm

Find the number of detected corner points in a given image using Harris Corner Detection Algorithm. Note that, Following criterion MUST be satisfied while applying Harris Corner detection Algorithm:

(a)  The size of neighbourhood considered for corner detection = 2.
(b)  Aperture parameter of Sobel derivative used = 3.
(c)  Harris detector free parameter in the equation = 0.04.

How many corners are detected?

1.   169
2.   222
3.   264
4.   309


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

%matplotlib inline

# Read in the image
image = cv2.imread('line.png')

# Make a copy of the image
image_copy = np.copy(image)

# Change color to RGB (from BGR)
image_copy = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)

###YOUR CODE STARTS HERE

## STEP 1:  Convert to grayscale 

## STEP 2: Detect corners 

## STEP 3: Dilate corner image to enhance corner points

## STEP 4:set threshold value as 0.1 * (maximum value of dilated corner image obtained from STEP3)

## STEP 5: Count numer of detected corner points and draw them on the image


### YOUR CODE ENDS HERE

# Part-2

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
import timeit
import unittest

## Please DONOT remove these lines. 
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

### Data Loading and Pre-processing

In [45]:
# check availability of GPU and set the device accordingly
#### YOUR CODE STARTS HERE ####
device = torch.device("cuda")
print(device)
print(torch.__version__)
#### YOUR CODE ENDS HERE ####

# define a transforms for preparing the dataset
transform = transforms.Compose([
        transforms.ToTensor(), # convert the image to a pytorch tensor
        transforms.Normalize((0.1307,), (0.3081,)) # normalise the images with mean and std of the dataset
        ])

cuda
1.6.0


In [30]:
import torchvision
# Load the MNIST training, test datasets using `torchvision.datasets.MNIST` using the transform defined above
#### YOUR CODE STARTS HERE ####
train_dataset = torchvision.datasets.MNIST(root = "./data", train = True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root = "./data", train = False, download=True, transform=transform)

print(len(train_dataset))
print(len(test_dataset))
print(train_dataset[0][0].size())
#### YOUR CODE ENDS HERE ####

60000
10000
torch.Size([1, 28, 28])


In [31]:
# create dataloaders for training and test datasets
# use a batch size of 32 and set shuffle=True for the training set
#### YOUR CODE STARTS HERE ####
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)
#### YOUR CODE ENDS HERE ####

*italicised text*### Network Definition

In [77]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #### YOUR CODE STARTS HERE ####
        # define a conv layer with output channels as 16, kernel size of 3 and stride of 1
        self.conv1 = nn.Conv2d(1, 16, 3, stride=1)
        # define a conv layer with output channels as 32, kernel size of 3 and stride of 1
        self.conv2 = nn.Conv2d(16, 32, 3, stride=1)
        # define a conv layer with output channels as 64, kernel size of 3 and stride of 1
        self.conv3 = nn.Conv2d(32, 64, 3, stride=1)
        # define a max pooling layer with kernel size 2
        self.maxpool = nn.MaxPool2d(2, stride=2)
        # define dropout layer with a probability of 0.25
        self.dropout1 = nn.Dropout(p=0.25)
        # define dropout layer with a probability of 0.5
        self.dropout2 = nn.Dropout(p=0.5)
        # define a linear(dense) layer with 128 output features
        self.fc1 = nn.Linear(in_features=64*11*11, out_features=128)
        # define a linear(dense) layer with output features corresponding to the number of classes in the dataset
        self.fc2 = nn.Linear(128,10)
        self.act = nn.ReLU()
        #### YOUR CODE ENDS HERE ####

    def forward(self, x):
        # Use the layers defined above in a sequential way (folow the same as the layer definitions above) and 
        # write the forward pass, after each of conv1, conv2, conv3 and fc1 use a relu activation. 
        #### YOUR CODE STARTS HERE ####
        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = self.act(self.conv3(x))
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.dropout2(x)
        x = x.reshape(x.size(0), -1)
        #print(x.size())
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        #### YOUR CODE ENDS HERE ####
        output = F.log_softmax(x, dim=1)
        return output

### Sanity Check
Make sure all the tests below pass without any errors, before you proceed with the training part.

In [78]:
import unittest

class TestImplementations(unittest.TestCase):
    
    # Dataloading tests
    def test_dataset(self):
        self.dataset_classes = ['0 - zero',
                                '1 - one',
                                '2 - two',
                                '3 - three',
                                '4 - four',
                                '5 - five',
                                '6 - six',
                                '7 - seven',
                                '8 - eight',
                                '9 - nine']
        self.assertTrue(train_dataset.classes == self.dataset_classes)
        self.assertTrue(train_dataset.train == True)
    
    def test_dataloader(self):        
        self.assertTrue(train_dataloader.batch_size == 32)
        self.assertTrue(test_dataloader.batch_size == 32)      
         
    def test_total_parameters(self):
        model = Net().to(device)
        self.assertTrue(sum(p.numel() for p in model.parameters()) == 1015946)

suite = unittest.TestLoader().loadTestsFromModule(TestImplementations())
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.012s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

### Training and Inference

In [79]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
      #### YOUR CODE STARTS HERE ####
        # send the image, target to the device
        data = data.to(device)
        target = target.to(device)
        # flush out the gradients stored in optimizer
        model.zero_grad()
        # pass the image to the model and assign the output to variable named output
        output = model(data)
        # calculate the loss (use nll_loss in pytorch)
        loss = nn.NLLLoss()(output, target)
        # do a backward pass
        loss.backward()
        # update the weights
        optimizer.step()
      #### YOUR CODE ENDS HERE ####
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [80]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
          ### YOUR CODE STARTS HERE ####
            # send the image, target to the device
            data = data.to(device)
            target = target.to(device)
            # pass the image to the model and assign the output to variable named output
            output = model(data)
            test_loss += nn.NLLLoss()(output, target).item()# sum up batch loss
          #### YOUR CODE ENDS HERE ####
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Question 4

Run the code cell below and report the final test accuracy (If you are not getting the exact number shown in options, please report the closest number).
1. 64%
2. 79%
3. 97%
4. 90%

In [81]:
model = Net().to(device)

## Define Adam Optimiser with a learning rate of 0.01
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

start = timeit.default_timer()
for epoch in range(1, 11):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305056
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.231873
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.434249
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.637274
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.788700
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.675811
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.430757
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.318939
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.474893
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.560169
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.197312
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.178759
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.217142
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.329004
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.137676
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.213552
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.442704
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.017484
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.283247
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.178546
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.480612
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.028242
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.244039
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.130371
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.087993
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.104751
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.091805
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.136555
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.069086
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.237662
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.109689
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.305009
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.184042
Train Epoch: 2 [53120/60000 (89%)]	Loss: 0.243332
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.128064
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.114030
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.004911
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.361643
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.203043
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.508823


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.059998
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.132503
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.011009
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.345349
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.310686
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.045014
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.106533
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.109893
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.059160
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.028243
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.071185
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.074010
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.095843
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.006811
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.031117
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.078685
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.278278
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.233883
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.300580
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.104295


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.045405
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.178479
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.207647
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.094118
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.010578
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.234718
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.305236
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.310392
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.165165
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.026506
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.038637
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.126243
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.146953
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.602720
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.019533
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.142435
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.096457
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.300078
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.057439
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.050196


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.373717
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.002585

Test set: Average loss: 0.0021, Accuracy: 9794/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.068043
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.130334
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.299257
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.230564
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.082306
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.003302
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.062820
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.154741
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.073521
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.020592
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.008547
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.442577
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.012897
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.398905
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.269867
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.454757
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.278816
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (73%)]	Loss: 0.006158
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.412834
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.076201
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.113036
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.052422
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.048426
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.337342
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.072418
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.084791
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.278159
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.064772
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.132108
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.405618
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.348251
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.460173
Train Epoch: 9 [53120/60000 (89%)]	Loss: 0.239527
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.084554
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.243898
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.215239
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.312440


### Question 5

Modify the network to replace ReLU activations with Sigmoid and report the final test accuracy by running the cell below. (If you are not getting the exact number shown in options, please report the closest number). 

1. 95%
2. 54%
3. 20%
4. 9%

In [84]:
class NetSigmoid(nn.Module):
    def __init__(self):
        super(NetSigmoid, self).__init__()
        #### YOUR CODE STARTS HERE ####
        # define a conv layer with output channels as 16, kernel size of 3 and stride of 1
        self.conv1 = nn.Conv2d(1, 16, 3, stride=1)
        # define a conv layer with output channels as 32, kernel size of 3 and stride of 1
        self.conv2 = nn.Conv2d(16, 32, 3, stride=1)
        # define a conv layer with output channels as 64, kernel size of 3 and stride of 1
        self.conv3 = nn.Conv2d(32, 64, 3, stride=1)
        # define a max pooling layer with kernel size 2
        self.maxpool = nn.MaxPool2d(2, stride=2)
        # define dropout layer with a probability of 0.25
        self.dropout1 = nn.Dropout(p=0.25)
        # define dropout layer with a probability of 0.5
        self.dropout2 = nn.Dropout(p=0.5)
        # define a linear(dense) layer with 128 output features
        self.fc1 = nn.Linear(in_features=64*11*11, out_features=128)
        # define a linear(dense) layer with output features corresponding to the number of classes in the dataset
        self.fc2 = nn.Linear(128,10)
        self.act = nn.Sigmoid()
        #### YOUR CODE ENDS HERE ####

    def forward(self, x):
        # Use the layers defined above in a sequential way (folow the same as the layer definitions above) and 
        # write the forward pass, after each of conv1, conv2, conv3 and fc1 use a relu activation. 
        #### YOUR CODE STARTS HERE ####
        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = self.act(self.conv3(x))
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.dropout2(x)
        x = x.reshape(x.size(0), -1)
        #print(x.size())
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        #### YOUR CODE ENDS HERE ####
        output = F.log_softmax(x, dim=1)
        return output

In [86]:
model = NetSigmoid().to(device)

## Define Adam Optimiser with a learning rate of 0.01
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

start = timeit.default_timer()

for epoch in range(1, 11):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)

stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.364542
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.367649
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.349978
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.507189
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.456176
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.294369
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.241853
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.268997
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.304546
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.361843
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.309218
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.371996
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.490387
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.361096
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.328178
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.355644
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.377111
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.299351
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.418519
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.427959
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 2.275963
Train Epoch: 2 [46080/60000 (77%)]	Loss: 2.415900
Train Epoch: 2 [46720/60000 (78%)]	Loss: 2.458112
Train Epoch: 2 [47360/60000 (79%)]	Loss: 2.340971
Train Epoch: 2 [48000/60000 (80%)]	Loss: 2.444527
Train Epoch: 2 [48640/60000 (81%)]	Loss: 2.392964
Train Epoch: 2 [49280/60000 (82%)]	Loss: 2.317478
Train Epoch: 2 [49920/60000 (83%)]	Loss: 2.439217
Train Epoch: 2 [50560/60000 (84%)]	Loss: 2.389455
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.384699
Train Epoch: 2 [51840/60000 (86%)]	Loss: 2.505224
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.335656
Train Epoch: 2 [53120/60000 (89%)]	Loss: 2.432651
Train Epoch: 2 [53760/60000 (90%)]	Loss: 2.411354
Train Epoch: 2 [54400/60000 (91%)]	Loss: 2.402754
Train Epoch: 2 [55040/60000 (92%)]	Loss: 2.332428
Train Epoch: 2 [55680/60000 (93%)]	Loss: 2.352187
Train Epoch: 2 [56320/60000 (94%)]	Loss: 2.345604
Train Epoch: 2 [56960/60000 (95%)]	Loss: 2.339038
Train Epoch: 2 [57600/60000 (96%)]	Loss: 2.372038


Train Epoch: 4 [29440/60000 (49%)]	Loss: 2.459793
Train Epoch: 4 [30080/60000 (50%)]	Loss: 2.359828
Train Epoch: 4 [30720/60000 (51%)]	Loss: 2.356207
Train Epoch: 4 [31360/60000 (52%)]	Loss: 2.354985
Train Epoch: 4 [32000/60000 (53%)]	Loss: 2.392312
Train Epoch: 4 [32640/60000 (54%)]	Loss: 2.343949
Train Epoch: 4 [33280/60000 (55%)]	Loss: 2.234704
Train Epoch: 4 [33920/60000 (57%)]	Loss: 2.393085
Train Epoch: 4 [34560/60000 (58%)]	Loss: 2.338331
Train Epoch: 4 [35200/60000 (59%)]	Loss: 2.375870
Train Epoch: 4 [35840/60000 (60%)]	Loss: 2.431165
Train Epoch: 4 [36480/60000 (61%)]	Loss: 2.290276
Train Epoch: 4 [37120/60000 (62%)]	Loss: 2.474497
Train Epoch: 4 [37760/60000 (63%)]	Loss: 2.319713
Train Epoch: 4 [38400/60000 (64%)]	Loss: 2.295508
Train Epoch: 4 [39040/60000 (65%)]	Loss: 2.310995
Train Epoch: 4 [39680/60000 (66%)]	Loss: 2.372563
Train Epoch: 4 [40320/60000 (67%)]	Loss: 2.353860
Train Epoch: 4 [40960/60000 (68%)]	Loss: 2.372604
Train Epoch: 4 [41600/60000 (69%)]	Loss: 2.464016


Train Epoch: 6 [14080/60000 (23%)]	Loss: 2.288362
Train Epoch: 6 [14720/60000 (25%)]	Loss: 2.302605
Train Epoch: 6 [15360/60000 (26%)]	Loss: 2.326597
Train Epoch: 6 [16000/60000 (27%)]	Loss: 2.355999
Train Epoch: 6 [16640/60000 (28%)]	Loss: 2.434383
Train Epoch: 6 [17280/60000 (29%)]	Loss: 2.299881
Train Epoch: 6 [17920/60000 (30%)]	Loss: 2.439094
Train Epoch: 6 [18560/60000 (31%)]	Loss: 2.326279
Train Epoch: 6 [19200/60000 (32%)]	Loss: 2.285989
Train Epoch: 6 [19840/60000 (33%)]	Loss: 2.376059
Train Epoch: 6 [20480/60000 (34%)]	Loss: 2.372981
Train Epoch: 6 [21120/60000 (35%)]	Loss: 2.288904
Train Epoch: 6 [21760/60000 (36%)]	Loss: 2.366858
Train Epoch: 6 [22400/60000 (37%)]	Loss: 2.238388
Train Epoch: 6 [23040/60000 (38%)]	Loss: 2.297058
Train Epoch: 6 [23680/60000 (39%)]	Loss: 2.339167
Train Epoch: 6 [24320/60000 (41%)]	Loss: 2.379823
Train Epoch: 6 [24960/60000 (42%)]	Loss: 2.311961
Train Epoch: 6 [25600/60000 (43%)]	Loss: 2.376903
Train Epoch: 6 [26240/60000 (44%)]	Loss: 2.306467


Train Epoch: 7 [58880/60000 (98%)]	Loss: 2.324890
Train Epoch: 7 [59520/60000 (99%)]	Loss: 2.235597

Test set: Average loss: 0.0736, Accuracy: 1135/10000 (11%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 2.341202
Train Epoch: 8 [640/60000 (1%)]	Loss: 2.307824
Train Epoch: 8 [1280/60000 (2%)]	Loss: 2.331468
Train Epoch: 8 [1920/60000 (3%)]	Loss: 2.328876
Train Epoch: 8 [2560/60000 (4%)]	Loss: 2.368043
Train Epoch: 8 [3200/60000 (5%)]	Loss: 2.418035
Train Epoch: 8 [3840/60000 (6%)]	Loss: 2.365932
Train Epoch: 8 [4480/60000 (7%)]	Loss: 2.378303
Train Epoch: 8 [5120/60000 (9%)]	Loss: 2.525214
Train Epoch: 8 [5760/60000 (10%)]	Loss: 2.354757
Train Epoch: 8 [6400/60000 (11%)]	Loss: 2.421330
Train Epoch: 8 [7040/60000 (12%)]	Loss: 2.477476
Train Epoch: 8 [7680/60000 (13%)]	Loss: 2.274465
Train Epoch: 8 [8320/60000 (14%)]	Loss: 2.338856
Train Epoch: 8 [8960/60000 (15%)]	Loss: 2.336114
Train Epoch: 8 [9600/60000 (16%)]	Loss: 2.344796
Train Epoch: 8 [10240/60000 (17%)]	Loss: 2.397911
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (73%)]	Loss: 2.246641
Train Epoch: 9 [44160/60000 (74%)]	Loss: 2.418995
Train Epoch: 9 [44800/60000 (75%)]	Loss: 2.338120
Train Epoch: 9 [45440/60000 (76%)]	Loss: 2.285479
Train Epoch: 9 [46080/60000 (77%)]	Loss: 2.400745
Train Epoch: 9 [46720/60000 (78%)]	Loss: 2.355004
Train Epoch: 9 [47360/60000 (79%)]	Loss: 2.220232
Train Epoch: 9 [48000/60000 (80%)]	Loss: 2.270728
Train Epoch: 9 [48640/60000 (81%)]	Loss: 2.337993
Train Epoch: 9 [49280/60000 (82%)]	Loss: 2.343009
Train Epoch: 9 [49920/60000 (83%)]	Loss: 2.571686
Train Epoch: 9 [50560/60000 (84%)]	Loss: 2.336145
Train Epoch: 9 [51200/60000 (85%)]	Loss: 2.448752
Train Epoch: 9 [51840/60000 (86%)]	Loss: 2.372742
Train Epoch: 9 [52480/60000 (87%)]	Loss: 2.390298
Train Epoch: 9 [53120/60000 (89%)]	Loss: 2.354712
Train Epoch: 9 [53760/60000 (90%)]	Loss: 2.449754
Train Epoch: 9 [54400/60000 (91%)]	Loss: 2.283339
Train Epoch: 9 [55040/60000 (92%)]	Loss: 2.403474
Train Epoch: 9 [55680/60000 (93%)]	Loss: 2.557542


### Question 6

Train the network defined in Question-1 with Adagrad optimizer (with the same learning rate). Report the final test accuracy by running the cell below. (If you are not getting the exact number shown in options, please report the closest number). 

1. 80%
2. 99%
3. 92%
4. 85%

In [87]:
model = Net().to(device)

## Define Adam Optimiser with a learning rate of 0.01
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

start = timeit.default_timer()

for epoch in range(1, 11):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)

stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332204
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.977420
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.524829
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.244753
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.494389
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.177629
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.186823
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.312735
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.159734
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.056032
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.060879
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.085933
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.112961
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.147647
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.246392
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.139998
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.707021
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.125164
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.181076
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.157195
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.289010
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.040319
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.282996
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.005827
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.028612
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.051713
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.173579
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.004092
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.014099
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.026722
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.043635
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.033345
Train Epoch: 2 [53120/60000 (89%)]	Loss: 0.094558
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.003432
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.002961
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.074039
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.046764
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.035697
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.017118
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.043658


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.054676
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.036110
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.016587
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.069368
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.061265
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.001609
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.001461
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.003491
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.017297
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.002739
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.150638
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.016196
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.088480
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.028721
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.011449
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.006926
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.001074
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.009889
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.012017
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.008089


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.005792
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.003056
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.099494
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.005860
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.077881
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.008342
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.011418
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.006524
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.005122
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.004805
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.002087
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.000667
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.073646
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.016290
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.001954
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.154613
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.065448
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.106915
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.012760
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.000705


Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.007504
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.000617
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.001485

Test set: Average loss: 0.0008, Accuracy: 9905/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.030567
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.002224
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.025302
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.002375
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.057834
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.007647
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.022310
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.000372
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.168288
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.011396
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.004397
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.048782
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.001122
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.065542
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.306253
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.146705
Train Epoch: 8 [10

Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.001314
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.007222
Train Epoch: 9 [43520/60000 (73%)]	Loss: 0.070818
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.070478
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.006677
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.268934
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.001339
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.023571
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.002338
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.007353
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.074535
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.011712
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.002284
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.001141
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.013122
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.008412
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.028442
Train Epoch: 9 [53120/60000 (89%)]	Loss: 0.016123
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.050572
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.134418


### Question 7

Initialize the `Conv2d` layers in the network defined in Question-1 `(Net)` with all ones (both weights and bias). Train the network with Adam optimizer and report the final test accuracy by running the cell below. (If you are not getting the exact number shown in options, please report the closest number). 

1. 11%
2. 18%
3. 97%
4. 6%

In [ ]:
model = Net().to(device)

def init_weights(m):
  #### YOUR CODE STARTS HERE ####
  if isinstance(m, nn.Conv2d):
        nn.init.ones_(m.weight.data)
        nn.init.ones_(m.bias.data)
  #### YOUR CODE ENDS HERE ####
  

model.apply(init_weights)  
## Define Adam Optimiser with a learning rate of 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

start = timeit.default_timer()

for epoch in range(1, 5):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)

stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )

Train Epoch: 1 [0/60000 (0%)]	Loss: 171407.203125
Train Epoch: 1 [640/60000 (1%)]	Loss: 16107.316406
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.155703
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.251598
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.307730
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.336328
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.308820
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303578
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.310734
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.299718
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.295515
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.304152
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.308152
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.278540
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.299958
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.312460
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.296121
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.305007
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.304249
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.295354
Train Epoch: 1 [1280

Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.317425
Train Epoch: 2 [45440/60000 (76%)]	Loss: 2.312946
Train Epoch: 2 [46080/60000 (77%)]	Loss: 2.294475
Train Epoch: 2 [46720/60000 (78%)]	Loss: 2.290356
Train Epoch: 2 [47360/60000 (79%)]	Loss: 2.289973
Train Epoch: 2 [48000/60000 (80%)]	Loss: 2.308160
Train Epoch: 2 [48640/60000 (81%)]	Loss: 2.287598
Train Epoch: 2 [49280/60000 (82%)]	Loss: 2.311833
Train Epoch: 2 [49920/60000 (83%)]	Loss: 2.306494
Train Epoch: 2 [50560/60000 (84%)]	Loss: 2.288367
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.308020
Train Epoch: 2 [51840/60000 (86%)]	Loss: 2.300387
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.279874
Train Epoch: 2 [53120/60000 (89%)]	Loss: 2.304740
Train Epoch: 2 [53760/60000 (90%)]	Loss: 2.313201
Train Epoch: 2 [54400/60000 (91%)]	Loss: 2.310272
Train Epoch: 2 [55040/60000 (92%)]	Loss: 2.308460
Train Epoch: 2 [55680/60000 (93%)]	Loss: 2.295084
Train Epoch: 2 [56320/60000 (94%)]	Loss: 2.294785
Train Epoch: 2 [56960/60000 (95%)]	Loss: 2.270759


### Question 8

Initialize the network defined in Question-1 `(Net)` with Xavier's initialization ([torch.nn.init.xavier_normal](https://pytorch.org/docs/stable/nn.init.html))(for bias use zero). Train the network with Adam optimizer and report the final test accuracy by running the cell below. (If you are not getting the exact number shown in options, please report the closest number). 


1. 88%
2. 74%
3. 97%
4. 80%

In [ ]:
model = Net().to(device)

def init_weights(m):
  #### YOUR CODE STARTS HERE ####
  pass
  #### YOUR CODE ENDS HERE #### 

model.apply(init_weights)  
## Define Adam Optimiser with a learning rate of 0.01
optimizer = 

start = timeit.default_timer()

for epoch in range(1, 5):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)

stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )

### Question 9

Add three batch-norm layers to the network defined in `Question-1` and report the final test accuracy by running the cell below. 

1. 92%
2. 89%
3. 98%
4. 74%

In [ ]:
class NetBatchNorm(nn.Module):
    def __init__(self):
        super(NetBatchNorm, self).__init__()
        #### YOUR CODE STARTS HERE ####
        # define a conv layer with output channels as 16, kernel size of 3 and stride of 1
        
        # define a batchnorm layer (2d) with 16 features
        
        # define a conv layer with output channels as 32, kernel size of 3 and stride of 1
        
        # define a batchnorm layer (2d) with 32 features
        
        # define a conv layer with output channels as 64, kernel size of 3 and stride of 1
        
        # define a batchnorm layer (2d) with 64 features
       
        # define dropout layer with a probability of 0.25
        
        # define dropout layer with a probability of 0.5
        
        # define a linear(dense) layer with 128 output features
        
        # self.bn1 = nn.BatchNorm2d(128)
        # define a linear(dense) layer with output features corresponding to the number of classes in the dataset
        
        #### YOUR CODE ENDS HERE ####

    def forward(self, x):
        #### YOUR CODE STARTS HERE ####
        x = 
        #### YOUR CODE ENDS HERE ####
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model = NetBatchNorm().to(device)
# If you implemented the batchnorm layers correctly below line should return true
sum(p.numel() for p in model.parameters()) == 1016170

In [ ]:


## Define Adam Optimiser with a learning rate of 0.01
optimizer = 

start = timeit.default_timer()
for epoch in range(1, 11):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )

# Part-3

This section is un-graded and purely for practice. 

Main focus of this part is to help you flex the deep learning muscles built in the above part. You should build a network on the [SVHN dataset](http://ufldl.stanford.edu/housenumbers/). This dataset is similar to MNIST but unlike MNIST, the images are colored and more complex. 

As of writing this, the state-of-the-art(SoTA) performance on this dataset is 98.98%. You can try to start with the simple network we defined above for the MNSIT dataset(with some modification for dealing with different sized colored images unlike MNIST). But to achive the SoTA performance you need to do a lot of hackery. These are list of few things, we would encourage you to try: 

- Use data augmentation wisely. Read and understand how to perform the augmentations listed below. 
    * RandomFlips, Color Jittering
    * Cutout, Cutmix
    * Mixup
    * Auto-augment

- Try to increase the image size using standard image interpolation techniques. Try using tricks like Progressive resizing of images and see if it helps. 

- After certain number of layers, adding more layer might not be of much help, run experiments on SVHN and see if you observe this. 

- To understand the difficulties in training deeper networks read this paper: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

- To improve the performance on SVHN, try using architectures like [ResNet](https://arxiv.org/abs/1512.03385), [DesnseNet](https://arxiv.org/abs/1608.06993) or [EfficientNet](https://arxiv.org/abs/1905.11946). Most of these architectures are available by default in PyTorch.
